<a href="https://colab.research.google.com/github/rlagosb/taller_eiv/blob/main/5_Ejercicio_ETL_(Python).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio:

Utilizando los datos de interconsultas del SSMN entre 2019 y 2023, cree un flujo de Extracción, Transformación y Carga para responder: ¿Cuál fue la demanda de interconsultas de Gastroenterología y Coloproctología por centro entre 2019 y 2024?


# Configurar área de trabajo y cargar datos
Usaremos las librerías de [Plotly](https://plotly.com/graphing-libraries/) para construir gráficos interactivos 📊✨.

In [ ]:
# importar librerías
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# definir carpetas de trabajo
datos_crudos_interconsultas = 'https://raw.githubusercontent.com/rlagosb/taller_eiv/main/datos_crudos/Solicitud%20Informaci%C3%B3n%20LT-AO008T00001801.parquet'
path_dw = 'https://raw.githubusercontent.com/rlagosb/taller_eiv/main/datos_multidimensionales/Data_Warehouse_Dims.xlsx'

# Extraer

Explique a continuación qué es lo que hace este bloque de código:

In [ ]:
sic_raw = pd.read_parquet(datos_crudos_interconsultas)
print(sic_raw.info())
sic_raw

# Transformar SIC

In [ ]:
sic = sic_raw.copy()
#generamos las columnas Año y Mes
sic['Año'] = sic['F_ENTRADA'].dt.year
sic['Mes'] = sic['F_ENTRADA'].dt.month

#primero agrupamos la base sic por sexo, prestación, establecimiento de destino, edad, año y mes
# ****FALTAN ALGUNAS COLUMNAS: COMPLETE COLUMNAS NECESARIAS REEMPLAZANDO LA LÍNEA !!!!!!!!!!!!****
sic = sic[['SEXO',_____________,'Año','Mes']].value_counts(sort=False).reset_index()
sic

# Crear cubo

In [ ]:
# Cargar dimensiones

# 1. Centros: Leer, seleccionar 3 columnas (Código, especialidad y nombre de la comuna) y renombrar campo Comuna
centros = pd.read_excel(path_dw, sheet_name='centros')[['Codigo','Nombre','Nombre Comuna']].rename(columns={'Nombre':'Centro','Nombre Comuna':'Comuna'})
# 2. Sexo: Leer hoja completa
sexo = pd.read_excel(path_dw, sheet_name='sexo')
# 3. Edad: Leer hoja completa
edad = pd.read_excel(path_dw, sheet_name='edad')
# 4. Especialidad: Leer y renombrar Nombre por Especialidad
especialidad = pd.read_excel(path_dw, sheet_name='prestacion').rename(columns={'Nombre':'Especialidad'})

¿Qué data frames cumplen las siguientes funciones en esta base de datos?
*   **Fact**= _________________
*   **Dimensión 1**= _________________
*   **Dimensión 2**= _________________
*   **Dimensión 3**= _________________
*   **Dimensión 4**= _________________



In [ ]:
#Generamos el cubo
cubo = (sic.rename(columns={'count':'Interconsultas'}). # 1. Renombrar 'count' a Interconsultas
        # 2. Merge con sexo
        merge(sexo, left_on='SEXO', right_on='Codigo', how='left').
        # 3. Merge con centros
        merge(centros, left_on='ESTAB_DEST', right_on='Codigo', how='left').
        # 4. Merge con edad
        merge(edad, left_on='EDAD', right_on='Edad', how='left').
        # 5. Merge con especialidad
        merge(especialidad, left_on='PRESTA_MIN', right_on='Codigo', how='left'))

# 7. Selección final de columnas
cubo[['Año','Mes','Especialidad','Centro','Comuna','Sexo','Edad','RangoEdad5','RangoEdad10','Interconsultas']]

# Graficar series de tiempo

In [ ]:
# Generamos una función que toma como parámetro el nombre de la serie:
# 'Consultas_producidas', 'Consultas_inasistencia', 'Oferta_consultas', 'Consultas_solicitadas', 'Lista_espera_inicial'

def graficar_serie(serie):

  df = cubo.groupby(['Año','Centro']).agg({serie:'sum'}).reset_index()

  # Generar el gráfico
  fig = px.line(df,
                x="Año",
                y=serie,
                color="Centro",
                title=serie+'  por Periodo y Centro')
  fig.show()

# Ejecutamos la función

graficar_serie('Interconsultas')

In [ ]:
# También podemos agregar un parámetro que permita filtrar por especialidad

def graficar_serie_especialidad(serie, especialidad):

  # generamos una variable para filtrar por especialidad
  filtro_especialidad = (cubo['Especialidad']==especialidad)
  df = cubo[filtro_especialidad].groupby(['Año','Centro']).agg({serie:'sum'}).reset_index()


  # Generar el gráfico
  titulo = serie+'  por Año y Centro ('+especialidad+')'
  fig = px.line(df,
                x="Año",
                y=serie,
                color="Centro",
                title=titulo)
  fig.show()

graficar_serie_especialidad('Interconsultas', 'OBSTETRICIA')

# Análisis

* ¿Cuál fue la demanda de interconsultas de Gastroenterología y Coloproctología por centro entre 2019 y 2023?
* ¿Cuál fue la demanda de interconsultas de Pediatría el 2023?
* ¿Qué otra pregunta podría responder utilizando esta base de datos y funciones presentadas aquí?
